In [2]:
%env CUDA_VISIBLE_DEVICES=0
%env TRANSFORMERS_CACHE=/mnt/LLM/hub
%env OMP_NUM_THREADS=16

import os
import sys
sys.path.insert(0, '..')

import time
import random
from tqdm.auto import trange
import ipynbname  # pip install ipynbname

import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers

from src.aq import QuantizedWeight, _reconstruct_weight  # see adjacent file (aq.py)
from src.utils import calc_avg_bits, get_mean_nbits_by_codebook  # see adjacent file (aq.py)

torch.set_num_threads(16)
torch.backends.cudnn.allow_tf32 = False
torch.backends.cuda.matmul.allow_tf32 = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_loading_dir = '/extra_disk_1/vahe1994/BRRR/layer10.self_attn.q_proj.input_activation.pt'
num_codebooks = 1
nbits_per_codebook = 14
out_group_size = 1
in_group_size = 8
batch_size = 16384
beam_size = 1
beam_search_epochs = 100
sparsity_regularizer = 0
print_frequency = 10
scale_nbits = 2    # 0 means no scales, 16 means no compression;


#for each group we store num_codebooks * nbits_per_codebook - bits 
# for W matrix  we store out_features*(in_features // group_size) * num_codebooks * nbits_per_codebook - bits 
# 1 codebook store codebook_size*group_size*16 - bit 
# all codebooks store num_codebooks* codebook_size*group_size*16 -bits
in_features, out_features  = 8192, 8192
estimated_bits_per_param = calc_avg_bits(
    num_codebooks, out_group_size, in_group_size, nbits_per_codebook, in_features, out_features, scale_nbits)
print("Estimated bits / param", estimated_bits_per_param)

env: CUDA_VISIBLE_DEVICES=0
env: TRANSFORMERS_CACHE=/mnt/LLM/hub
env: OMP_NUM_THREADS=16
Estimated bits / param 2.0390625


In [3]:
import wandb

os.environ["WANDB_NOTEBOOK_NAME"] = os.path.join(os.getcwd(), ipynbname.name() + ".ipynb")

# start a new wandb run to track this script
run = wandb.init(
    # set the wandb project where this run will be logged
    dir=os.getcwd(),
    project="AddQuantization",
    entity = "rock-and-roll",
    save_code=True,
    name = f"{ipynbname.name()}_AQ_{num_codebooks=}_{out_group_size=}_{in_group_size=}_{nbits_per_codebook=}_{beam_search_epochs=}",
    settings=wandb.Settings(code_dir="."),
    # track hyperparameters and run metadata
    config={
    "num_codebooks" : num_codebooks,
    "out_group_size": out_group_size,
    "in_group_size": in_group_size,
    "group_size" : out_group_size * in_group_size,
    "batch_size" : batch_size,
    "beam_size" : beam_size,
    "nbits_per_codebook" : nbits_per_codebook,
    "Avg_bits": estimated_bits_per_param,
    "beam_search_epochs": beam_search_epochs,
    "sparsity_regularizer": sparsity_regularizer,
    "scale_nvits": scale_nbits,
    }
)
run.log({"Avg_bits": estimated_bits_per_param})

wandb: Currently logged in as: justheuristic (rock-and-roll). Use `wandb login --relogin` to force relogin


wandb: WARNING No relevant files were detected in the specified directory. No code will be logged to your run.


In [4]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-70b-hf", torch_dtype='auto', low_cpu_mem_usage=True)

X = torch.load(input_loading_dir, map_location='cpu').float().flatten(0, -2)
reference_weight = model.model.layers[10].self_attn.q_proj.weight.detach().to(device).float()

XTX = torch.zeros(X.shape[-1], X.shape[-1], device=device, dtype=torch.float64)
for i in range(0, len(X), batch_size):
    x_batch = X[i: i + batch_size].cuda().double()
    XTX.addmm_(x_batch.T, x_batch, alpha=1/len(X))
    del x_batch
XTX = XTX.float()
del X

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
quantized_layer = QuantizedWeight(
    reference_weight=reference_weight, num_codebooks=num_codebooks,
    nbits_per_codebook=nbits_per_codebook, scale_nbits=scale_nbits,
    out_group_size=out_group_size, in_group_size=in_group_size,
    max_iter=100,   # faster init, not tested
    verbose=True
)
opt = torch.optim.Adam(quantized_layer.parameters(), lr=1e-4, betas=(0.0, 0.95), amsgrad=True)

for epoch in range(10_000):
    start = time.perf_counter()
    reconstructed_weight = quantized_weight()
    delta_weight = (reconstructed_weight - reference_weight).double()
    loss = (delta_weight @ XTX.double()).flatten() @ delta_weight.flatten() / len(delta_weight)
    opt.zero_grad()
    loss.backward()
    opt.step()
    
    run.log({'loss':loss.item()}, step=epoch)
    
    if epoch % print_frequency == 0:
        print(f"loss={loss.item():.10f}\t",
              f"time_on_epoch {epoch} = {time.perf_counter() - start}")
    if (epoch + 1) % beam_search_epochs == 0:
        quantized_layer.requantize_(
            XTX, reference_weight,
            beam_size=beam_size,
            sparsity_regularizer=sparsity_regularizer,  # tip: use const_hparam * quantized_layer.codes.numel()
            dim_rng=random.Random(), verbose=True)

        if sparsity_regularizer != 0:
            sparsity_rate = ((quantized_layer.codes == 0).sum() / quantized_layer.codes.numel()).item()
            print(f"Sparsity rate {sparsity_rate:.5f}")
            run.log({'sparsity rate': sparsity_rate}, step=epoch)
            mean_code_nbits = sum(get_mean_nbits_by_codebook(quantized_layer.codes)) / num_codebooks
            print(f"mean_code_nbits {mean_code_nbits:.5f}")
            run.log({'Mean codebook length nbits': mean_code_nbits}, step=epoch)
            if in_group_size > 1 and out_group_size > 1:
                curr_avg_bits  = calc_avg_bits(num_codebooks, 1, mean_code_nbits,
                                     nbits_per_codebook, in_features, out_features, scale_nbits)
                run.log({"Avg_bits": curr_avg_bits}, step=epoch)

/home/jheuristic/GPTAQ_scales/Notebooks/../src/kmeans_1d.py:160: UserWarning: torch.searchsorted(): boundary tensor is non-contiguous, this will lower the performance due to extra data copy when converting non-contiguous tensor to contiguous, please use contiguous boundary tensor if possible. This message will only appear once per program. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/BucketizationUtils.h:38.)
  groupwise_cluster_indices = torch.searchsorted(border_indices[:, 1:], groupwise_ranks_1based, side='left')


initializing with kmeans:   0%|          | 0/1 [00:00<?, ?it/s]

/home/jheuristic/GPTAQ_scales/Notebooks/../src/aq.py:480: UserWarning: index_reduce() is in beta and the API may change at any time. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cuda/Indexing.cu:1193.)
  codebook_i, codes_i, reconstructed_weight_i = fit_kmeans(weight_residue, k=codebook_size, **kwargs)


loss=0.0258857615	 time_on_epoch 0 = 1.794874879065901
loss=0.0214160136	 time_on_epoch 10 = 0.13309287303127348
loss=0.0179490415	 time_on_epoch 20 = 0.13319369300734252
loss=0.0152740230	 time_on_epoch 30 = 0.13324946200009435
loss=0.0132520099	 time_on_epoch 40 = 0.1333391530206427
loss=0.0117486446	 time_on_epoch 50 = 0.13315503299236298
loss=0.0106320357	 time_on_epoch 60 = 0.13317235291469842
loss=0.0097964607	 time_on_epoch 70 = 0.13332686305511743
loss=0.0091640784	 time_on_epoch 80 = 0.13326736295130104
loss=0.0086792293	 time_on_epoch 90 = 0.13343094300944358


  0%|          | 0/1024 [00:00<?, ?it/s]

...

loss=0.0021539355	 time_on_epoch 2000 = 0.13354079297278076
loss=0.0021537524	 time_on_epoch 2010 = 0.13318546302616596
loss=0.0021536233	 time_on_epoch 2020 = 0.13371747301425785
loss=0.0021535109	 time_on_epoch 2030 = 0.13312378292903304
loss=0.0021534079	 time_on_epoch 2040 = 0.13319656299427152
loss=0.0021533111	 time_on_epoch 2050 = 0.13315940299071372
loss=0.0021532188	 time_on_epoch 2060 = 0.13317721302155405
loss=0.0021531298	 time_on_epoch 2070 = 0.13319819292519242
loss=0.0021530437	 time_on_epoch 2080 = 0.13323167408816516
loss=0.0021529600	 time_on_epoch 2090 = 0.13312618399504572


  0%|          | 0/1024 [00:00<?, ?it/s]

loss=0.0021505469	 time_on_epoch 2100 = 0.13366506597958505
loss=0.0021503773	 time_on_epoch 2110 = 0.13322416704613715
loss=0.0021502564	 time_on_epoch 2120 = 0.13312692707404494
loss=0.0021501509	 time_on_epoch 2130 = 0.13316607708111405
loss=0.0021500539	 time_on_epoch 2140 = 0.1331694460241124
loss=0.0021499626	 time_on_epoch 2150 = 0.13314664701465517
loss=0.0021498754	 time_on_epoch 2160 = 0.13316499697975814
loss=0.0021497913	 time_on_epoch 2170 = 0.1332580370362848
loss=0.0021497098	 time_on_epoch 2180 = 0.13312629703432322
loss=0.0021496306	 time_on_epoch 2190 = 0.13338325603399426


  0%|          | 0/1024 [00:00<?, ?it/s]

loss=0.0021474483	 time_on_epoch 2200 = 0.1337250379147008
